In [1]:
from __future__ import unicode_literals, print_function

import random
from spacy import displacy

from spacy.lang.pt_br import Portuguese
from spacy.gold import GoldParse, biluo_tags_from_offsets

In [2]:
def reformat_train_data(tokenizer, examples):
    """Reformat data to match JSON format"""
    output = []
    for i, (text, entity_offsets) in enumerate(examples):
        doc = tokenizer(text)
        ner_tags = biluo_tags_from_offsets(tokenizer(text), entity_offsets)
        words = [w.text for w in doc]
        tags = ['-'] * len(doc)
        heads = [0] * len(doc)
        deps = [''] * len(doc)
        sentence = (range(len(doc)), words, tags, heads, deps, ner_tags)
        output.append((text, [(sentence, [])]))
    return output

In [3]:
def train(model_dir=None):
    train_data = [
        (
            'vocês trabalham com médico do trabalho?',
            [(len('vocês trabalham com '), len('vocês trabalham com médico do trabalho'), 'MEDICO')]
        ),
        (
            'quando tem gastroenterologista?',
            [(len('quando tem '), len('quando tem gastroenterologista'), 'MEDICO')]
        ),
        (
            'quando será que terá urolugista?',
            [(len('quando será que terá '), len('quando será que terá urolugista'), 'MEDICO')]
        ),
        (
            'quando tem médico otorrinolaringologista?',
            [(len('quando tem médico '), len('quando tem médico otorrinolaringologista'), 'MEDICO')]
        ),
        (
            'tem ortopedista?',
            [(len('tem '), len('tem ortopedista'), 'MEDICO')]
        ),
        (
            'quando tem médico angiologista aí?',
            [(len('quando tem médico '), len('quando tem médico angiologista'), 'MEDICO')]
        ),
        (
            'tem algum médico cardiologista atendendo essa semana?',
            [(len('tem algum médico '), len('tem algum médico cardiologista'), 'MEDICO')]
        ),
        (
            'vocês tem médico dermatologista?',
            [(len('vocês tem médico '), len('vocês tem médico dermatologista'), 'MEDICO')]
        ),
        (
            'qual o valor da consulta do doutor honório?',
            [(len('qual o valor da consulta do doutor '), len('qual o valor da consulta do doutor honório'), 'MEDICO')]
        ),
        (
            'quais os dias que doutor honório atende?',
            [(len('quais os dias que doutor '), len('quais os dias que doutor honório'), 'MEDICO')]
        )
    ]
    nlp = Portuguese(pipeline=['tensorizer', 'ner'])
    get_data = lambda: reformat_train_data(nlp.tokenizer, train_data)
    optimizer = nlp.begin_training(get_data)
    for itn in range(100):
        random.shuffle(train_data)
        losses = {}
        for raw_text, entity_offsets in train_data:
            doc = nlp.make_doc(raw_text)
            gold = GoldParse(doc, entities=entity_offsets)
            nlp.update(
                [doc], # Batch of Doc objects
                [gold], # Batch of GoldParse objects
                drop=0.5, # Dropout -- make it harder to memorise data
                sgd=optimizer, # Callable to update weights
                losses=losses)
#         print(losses)
    print("Save to", model_dir)
    nlp.to_disk(model_dir)

# Training

In [4]:
train('ner model')

Save to ner model


# Predicting

In [5]:
from IPython.core.display import display, HTML

def predict(model_dir=None, text=None):
    nlp = Portuguese(pipeline=['tensorizer', 'ner'])
    nlp.from_disk(model_dir)
    
    print("Load from", model_dir)

    doc = nlp(text)
    html = displacy.render(doc, style='ent')
    return display(HTML(html))

In [9]:
predict('ner model', 'Tem médico otorrino amanhã?')

Load from ner model
